In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 15.3 MB/s eta 0:00:00


In [4]:
import multiprocessing
import nltk
import string
from re import sub
from unidecode import unidecode
from gensim.models import Word2Vec
from collections import defaultdict
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.models.phrases import Phrases, Phraser
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from nltk.corpus import stopwords
from nltk.stem.porter import *

In [34]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [31]:
from nltk.tokenize import word_tokenize

Opening File...

In [6]:
# PATH = "D:/Program Files (x86)/Jupyter Projects/"
# filepath = PATH + "forumposts.csv"
# df = pd.read_csv(filepath)
df = pd.read_csv('forumposts.csv')
df

,author,date,post
0,OnnMacMahal,"Aug 30 2022,","\r\n\t\t\t\t\t,\r\nHi everyone, I’m OnnMacMaha..."
1,Joedangod,"Aug 30 2022,",\r\n\t\t\t\t\tYou're deleting the classic supp...
2,NuclearTonic,"Aug 30 2022,",\r\n\t\t\t\t\tI think this is a great thought!...
3,Detective0,"Aug 30 2022,",\r\n\t\t\t\t\tI'm worrying that this will decr...
4,Maedol,"Aug 30 2022,",\r\n\t\t\t\t\tWhy can't anthems be made availa...
...,...,...,...
193,infinitewhimsy,"Yesterday,",\r\n\t\t\t\t\tPosting feedback in this thread ...
194,Carywary,"Today,",\r\n\t\t\t\t\tI was ready to be super upset th...
195,hillard1959,"Today,","\r\n\t\t\t\t\t,\n,\n,Well, DPS is better. By a..."
196,infinitewhimsy,"Today,","\r\n\t\t\t\t\t,\n,\n,If anything I imagine lan..."


In [12]:
df['date'][185]

'Sep 06 2022,\xa0'

Correcting Dates...

In [13]:
df.loc[df['date'].str.contains('Yesterday'), 'date'] = 'Sep 07 2022'
df.loc[df['date'].str.contains('Today'), 'date'] = 'Sep 08 2022'
df

,author,date,post
0,OnnMacMahal,"Aug 30 2022,","\r\n\t\t\t\t\t,\r\nHi everyone, I’m OnnMacMaha..."
1,Joedangod,"Aug 30 2022,",\r\n\t\t\t\t\tYou're deleting the classic supp...
2,NuclearTonic,"Aug 30 2022,",\r\n\t\t\t\t\tI think this is a great thought!...
3,Detective0,"Aug 30 2022,",\r\n\t\t\t\t\tI'm worrying that this will decr...
4,Maedol,"Aug 30 2022,",\r\n\t\t\t\t\tWhy can't anthems be made availa...
...,...,...,...
193,infinitewhimsy,Sep 07 2022,\r\n\t\t\t\t\tPosting feedback in this thread ...
194,Carywary,Sep 08 2022,\r\n\t\t\t\t\tI was ready to be super upset th...
195,hillard1959,Sep 08 2022,"\r\n\t\t\t\t\t,\n,\n,Well, DPS is better. By a..."
196,infinitewhimsy,Sep 08 2022,"\r\n\t\t\t\t\t,\n,\n,If anything I imagine lan..."


In [18]:
df['date'] = df['date'].apply(lambda x: x[:11])
df

,author,date,post
0,OnnMacMahal,Aug 30 2022,"\r\n\t\t\t\t\t,\r\nHi everyone, I’m OnnMacMaha..."
1,Joedangod,Aug 30 2022,\r\n\t\t\t\t\tYou're deleting the classic supp...
2,NuclearTonic,Aug 30 2022,\r\n\t\t\t\t\tI think this is a great thought!...
3,Detective0,Aug 30 2022,\r\n\t\t\t\t\tI'm worrying that this will decr...
4,Maedol,Aug 30 2022,\r\n\t\t\t\t\tWhy can't anthems be made availa...
...,...,...,...
193,infinitewhimsy,Sep 07 2022,\r\n\t\t\t\t\tPosting feedback in this thread ...
194,Carywary,Sep 08 2022,\r\n\t\t\t\t\tI was ready to be super upset th...
195,hillard1959,Sep 08 2022,"\r\n\t\t\t\t\t,\n,\n,Well, DPS is better. By a..."
196,infinitewhimsy,Sep 08 2022,"\r\n\t\t\t\t\t,\n,\n,If anything I imagine lan..."


Removing Escape Secquences...

In [19]:
df = df.replace('[\\r,\\t,\\n,‘,’]','',regex=True)

In [20]:
df['post'][0]

'Hi everyone Im OnnMacMahal and I want to talk to you about the upcoming changes to the Minstrel class. Weve converted the minstrels Protector of Song specialization tree into a passive tree functioning the same way as the brawlers yellow tree does. Im here to answer a few questions up front and give you a bit of insight into our thinking regarding these changes.With U33.2 minstrels will no longer be able to choose the Protector of Song trait tree as their specialization. Youll still be able to specialize in the other two trait trees and the yellow tree isnt going anywhere! While you wont be able to choose yellow as your specialization it is still full of exciting stuff. For most players this means youll be able to access unique skills like Song of Aid and Call to Greatness while specializing in either The Watcher of Resolve or The Warrior-Skald trait lines.The yellow minstrel line has a strong identity but while there are good reasons to specialize in the Protector of Song those reaso

Generating corpus...

In [21]:
corpus = df[['post']].copy()
corpus

,post
0,Hi everyone Im OnnMacMahal and I want to talk ...
1,You're deleting the classic support bard arche...
2,I think this is a great thought! This really h...
3,I'm worrying that this will decrease the varie...
4,Why can't anthems be made available outside of...
...,...
193,Posting feedback in this thread is like some k...
194,I was ready to be super upset that my beloved ...
195,Well DPS is better. By a wide margin I think. ...
196,If anything I imagine landscape is the most af...


# Alternative: Generating Sentiment via CardiffNLP-Roberta_base Sentiment Classifier

In [38]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 18.1 MB/s eta 0:00:00


In [39]:
df2 = df.copy()

In [41]:
from transformers import pipeline

model_path = 'cardiffnlp/twitter-roberta-base-sentiment-latest'

sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [52]:
sentiment_task(df2['post'][0][:1024])[0]['label']

'neutral'

In [53]:
def new_sentiment(text):
  return sentiment_task(text[:1024])[0]['label'] 

In [54]:
df2['sentiment'] = df2['post'].apply(new_sentiment)

In [55]:
df2

,author,date,post,sentiment
0,OnnMacMahal,Aug 30 2022,Hi everyone Im OnnMacMahal and I want to talk ...,neutral
1,Joedangod,Aug 30 2022,You're deleting the classic support bard arche...,negative
2,NuclearTonic,Aug 30 2022,I think this is a great thought! This really h...,positive
3,Detective0,Aug 30 2022,I'm worrying that this will decrease the varie...,negative
4,Maedol,Aug 30 2022,Why can't anthems be made available outside of...,neutral
...,...,...,...,...
193,infinitewhimsy,Sep 07 2022,Posting feedback in this thread is like some k...,negative
194,Carywary,Sep 08 2022,I was ready to be super upset that my beloved ...,negative
195,hillard1959,Sep 08 2022,Well DPS is better. By a wide margin I think. ...,positive
196,infinitewhimsy,Sep 08 2022,If anything I imagine landscape is the most af...,neutral


In [56]:
df2.to_csv('forumposts_roberta.csv', index=False)

Removing stopwords...

In [60]:
df2['post'] = df['post']

In [62]:
stpwrds = stopwords.words('english') + list(string.punctuation)

In [63]:
def remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens_nosw = [word for word in tokens if not word.lower() in stpwrds]
    return " ".join(word for word in tokens_nosw)

In [64]:
df2['post'] = df2['post'].apply(remove_stopwords)

In [65]:
df2

,author,date,post,sentiment
0,OnnMacMahal,Aug 30 2022,Hi everyone Im OnnMacMahal want talk upcoming ...,neutral
1,Joedangod,Aug 30 2022,'re deleting classic support bard archetype re...,negative
2,NuclearTonic,Aug 30 2022,think great thought really helped define Brawl...,positive
3,Detective0,Aug 30 2022,'m worrying decrease variety healing choices m...,negative
4,Maedol,Aug 30 2022,ca n't anthems made available outside combat S...,neutral
...,...,...,...,...
193,infinitewhimsy,Sep 07 2022,Posting feedback thread like kind monkey 's pa...,negative
194,Carywary,Sep 08 2022,ready super upset beloved minstrel knife round...,negative
195,hillard1959,Sep 08 2022,Well DPS better wide margin think n't know gro...,positive
196,infinitewhimsy,Sep 08 2022,anything imagine landscape affected PC stun fe...,neutral


In [66]:
df2['post'] = df2['post'].str.split()
df2 = df2.explode('post')

In [67]:
df2 = df2.reset_index()
df2

,index,author,date,post,sentiment
0,0,OnnMacMahal,Aug 30 2022,Hi,neutral
1,0,OnnMacMahal,Aug 30 2022,everyone,neutral
2,0,OnnMacMahal,Aug 30 2022,Im,neutral
3,0,OnnMacMahal,Aug 30 2022,OnnMacMahal,neutral
4,0,OnnMacMahal,Aug 30 2022,want,neutral
...,...,...,...,...,...
20295,197,hillard1959,Sep 08 2022,else,negative
20296,197,hillard1959,Sep 08 2022,change,negative
20297,197,hillard1959,Sep 08 2022,change,negative
20298,197,hillard1959,Sep 08 2022,'s,negative


In [68]:
df2.to_csv('forumwords_forwc.csv', index=False)

# Previous approach

Creating world list...

In [22]:
def text_to_word_list(text, remove_polish_letters):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    text = remove_polish_letters(text)
    text = str(text)
    text = text.lower()

    # Clean the text
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ! ", text)
    text = sub(r"\?", " ? ", text)
    text = sub(r":", " : ", text)
    text = sub(r"\s{2,}", " ", text)

    text = text.split()

    return text  

In [23]:
corpus['post'] = corpus['post'].apply(lambda x: text_to_word_list(x, unidecode))

In [24]:
corpus

,post
0,"[hi, everyone, im, onnmacmahal, and, i, want, ..."
1,"[you're, deleting, the, classic, support, bard..."
2,"[i, think, this, is, a, great, thought, !, thi..."
3,"[i'm, worrying, that, this, will, decrease, th..."
4,"[why, can't, anthems, be, made, available, out..."
...,...
193,"[posting, feedback, in, this, thread, is, like..."
194,"[i, was, ready, to, be, super, upset, that, my..."
195,"[well, dps, is, better, by, a, wide, margin, i..."
196,"[if, anything, i, imagine, landscape, is, the,..."


Generating sentences...

In [25]:
file_model = corpus.copy()
file_model = file_model[file_model['post'].str.len()>1]

In [26]:
sent = [row for row in file_model['post']]

In [27]:
sent

[['hi',
  'everyone',
  'im',
  'onnmacmahal',
  'and',
  'i',
  'want',
  'to',
  'talk',
  'to',
  'you',
  'about',
  'the',
  'upcoming',
  'changes',
  'to',
  'the',
  'minstrel',
  'class',
  'weve',
  'converted',
  'the',
  'minstrels',
  'protector',
  'of',
  'song',
  'specialization',
  'tree',
  'into',
  'a',
  'passive',
  'tree',
  'functioning',
  'the',
  'same',
  'way',
  'as',
  'the',
  'brawlers',
  'yellow',
  'tree',
  'does',
  'im',
  'here',
  'to',
  'answer',
  'a',
  'few',
  'questions',
  'up',
  'front',
  'and',
  'give',
  'you',
  'a',
  'bit',
  'of',
  'insight',
  'into',
  'our',
  'thinking',
  'regarding',
  'these',
  'changes',
  'with',
  'u33',
  '2',
  'minstrels',
  'will',
  'no',
  'longer',
  'be',
  'able',
  'to',
  'choose',
  'the',
  'protector',
  'of',
  'song',
  'trait',
  'tree',
  'as',
  'their',
  'specialization',
  'youll',
  'still',
  'be',
  'able',
  'to',
  'specialize',
  'in',
  'the',
  'other',
  'two',
  'tra

Getting stopwords...

In [28]:
stpwrds = stopwords.words('english') + list(string.punctuation)
stpwrds

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

Stemming and removing stopwords...

In [ ]:
stemmer = PorterStemmer()

In [ ]:
processed_sent = []

In [ ]:
for s in sent:
    temp = [stemmer.stem(w.lower()) for w in s if w.lower() not in stpwrds]
    processed_sent.append(temp)

In [ ]:
processed_sent

[['hi',
  'everyon',
  'im',
  'onnmacmah',
  'want',
  'talk',
  'upcom',
  'chang',
  'minstrel',
  'class',
  'weve',
  'convert',
  'minstrel',
  'protector',
  'song',
  'special',
  'tree',
  'passiv',
  'tree',
  'function',
  'way',
  'brawler',
  'yellow',
  'tree',
  'im',
  'answer',
  'question',
  'front',
  'give',
  'bit',
  'insight',
  'think',
  'regard',
  'chang',
  'u33',
  '2',
  'minstrel',
  'longer',
  'abl',
  'choos',
  'protector',
  'song',
  'trait',
  'tree',
  'special',
  'youll',
  'still',
  'abl',
  'special',
  'two',
  'trait',
  'tree',
  'yellow',
  'tree',
  'isnt',
  'go',
  'anywher',
  'wont',
  'abl',
  'choos',
  'yellow',
  'special',
  'still',
  'full',
  'excit',
  'stuff',
  'player',
  'mean',
  'youll',
  'abl',
  'access',
  'uniqu',
  'skill',
  'like',
  'song',
  'aid',
  'call',
  'great',
  'special',
  'either',
  'watcher',
  'resolv',
  'warrior',
  'skald',
  'trait',
  'line',
  'yellow',
  'minstrel',
  'line',
  'strong'

Creating phrases and bigrams from the processed sentences...

In [ ]:
phrases = Phrases(processed_sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = phrases[processed_sent]
sentences[1]

['delet',
 'classic',
 'support',
 'bard',
 'archetyp',
 'remov',
 'closest',
 'thing',
 'game',
 'rcaptain',
 'competitor',
 'outsid',
 'mayb',
 'blm',
 'new',
 'crowthi',
 'yellow',
 'definit',
 'problemat',
 'realli',
 'feel_like',
 'gone',
 'rout',
 'make_competit',
 'non',
 'stack',
 'replac',
 'red_captain',
 'group_content',
 'rather',
 'leav',
 'us',
 'red_captain',
 'support',
 'domin',
 "that'",
 'exist',
 'decad']

Creating Word2Vec model...

In [ ]:
w2v_model = Word2Vec(min_count=2,
                     window=4,
                     vector_size=64,
                     sample=1e-5, 
                     alpha=0.04,
                     sg=1, 
                     min_alpha=0.0008, 
                    #  negative=7,
                     hs=1,
                     seed=123,
                     epochs=10,
                     workers=multiprocessing.cpu_count()-1)

w2v_model.build_vocab(sentences, progress_per=50000)

In [ ]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(65437, 512820)

In [ ]:
w2v_model.init_sims(replace=True)

C:\Users\sixbo\AppData\Local\Temp\ipykernel_20464\514372312.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [ ]:
w2v_model.save(PATH+"minifeed_word2vec.model")

Exporting preprocessed dataset for future uses...

In [ ]:
file_export = file_model.copy()
file_export['old_post'] = file_export['post']
file_export['old_post'] = file_export['old_post'].str.join(' ')
file_export['post'] = file_export['post'].apply(lambda x: ' '.join(bigram[x]))

In [ ]:
file_export[['post']].to_csv(PATH+'cleaned_forumposts.csv', index=False)

Getting word vectors...

In [ ]:
word_vectors = Word2Vec.load(PATH+"minifeed_word2vec.model").wv

Clustering...

In [ ]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))

d:\Anaconda\envs\tfenv\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=8.
  warnings.warn(


In [ ]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

[('translat', 0.994744598865509),
 ('warrior_skald', 0.9935766458511353),
 ('may_may', 0.9933280348777771),
 ('hard_tell', 0.9931867122650146),
 ('endgam', 0.9930075407028198),
 ("i'm_wait", 0.99193274974823),
 ('landscap_quest', 0.9909868240356445),
 ('play_around', 0.9905813932418823),
 ('sometim', 0.9904405474662781),
 ('pars', 0.9904268980026245)]

In [ ]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [ ]:
words = pd.DataFrame(word_vectors.index_to_key)
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [ ]:
upper_limit = 200
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: upper_limit if (model.transform([x.vectors]).min() == 0) else 1/(model.transform([x.vectors]).min()), axis=1)

In [ ]:
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [ ]:
words

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,anthem,"[-0.233034, -0.11393154, 0.017351234, 0.051178...",1,1,4.197139,4.197139
1,chang,"[-0.2053889, -0.11933444, 0.070904665, 0.06139...",0,-1,3.564504,-3.564504
2,use,"[-0.16143413, -0.14628202, 0.088525794, 0.0383...",0,-1,3.691172,-3.691172
3,heal,"[-0.18934107, -0.12375026, 0.052837644, 0.0617...",0,-1,4.247178,-4.247178
4,mini,"[-0.193201, -0.15239275, 0.06662942, 0.0476960...",0,-1,3.224910,-3.224910
...,...,...,...,...,...,...
2016,struggl,"[-0.20369641, -0.100541666, 0.052207083, 0.051...",0,-1,8.338169,-8.338169
2017,57,"[-0.18620099, -0.120482385, 0.040745255, 0.057...",0,-1,7.026922,-7.026922
2018,plu_3,"[-0.18553269, -0.07557886, 0.08070111, 0.02890...",0,-1,4.326411,-4.326411
2019,rel,"[-0.17368023, -0.068128854, 0.023189861, 0.077...",0,-1,4.425767,-4.425767


In [ ]:
words.iloc[np.where(words['cluster_value'] == 1)]

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,anthem,"[-0.233034, -0.11393154, 0.017351234, 0.051178...",1,1,4.197139,4.197139
7,buff,"[-0.18667737, -0.13465667, 0.041434553, 0.1026...",1,1,3.641694,3.641694
16,melodi,"[-0.20418124, -0.111128576, 0.020449653, 0.113...",1,1,3.716547,3.716547
25,effect,"[-0.20817116, -0.16178115, 0.035078708, 0.1087...",1,1,5.745622,5.745622
40,coda,"[-0.16839214, -0.1674611, 0.07941345, 0.089376...",1,1,4.545732,4.545732
...,...,...,...,...,...,...
1988,hey,"[-0.20132683, -0.18470962, 0.03757738, 0.07339...",1,1,5.861146,5.861146
1989,accept_chang,"[-0.168722, -0.1325762, 0.057916176, 0.1084844...",1,1,4.485581,4.485581
1998,anyon_even,"[-0.15415995, -0.1162383, 0.06377886, 0.068225...",1,1,4.057824,4.057824
1999,soa,"[-0.15155078, -0.17554978, 0.06923224, 0.10862...",1,1,6.547037,6.547037


In [ ]:
words.iloc[np.where(words['words'] == 'nerf')]

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
36,nerf,"[-0.17865908, -0.11527409, 0.0731994, 0.031257...",0,-1,4.943368,-4.943368


In [ ]:
words.iloc[np.where(words['words'] == 'buff')]

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
7,buff,"[-0.18667737, -0.13465667, 0.041434553, 0.1026...",1,1,3.641694,3.641694


Corpus sentiment...

In [ ]:
words['sentiment_coeff'].sum()

-8055.878687742656

In [ ]:
words['closeness_score'].sum()

11034.23440955142

Calculating each document's sentiment...

In [ ]:
corpus.iloc[np.where(corpus['post'].str.len() < 5)] # finding post with too little data for sentiment

,post
9,[nope]


In [ ]:
corpus['sentiment'] = 0.0

In [ ]:
corpus = corpus.drop(index=9)

In [ ]:
sentiment_arr = np.empty([197],dtype=float)

In [ ]:
words.iloc[np.where('delet' ==  words['words'])]

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
1270,delet,"[-0.19071999, -0.18309516, 0.07229778, 0.07439...",1,1,4.835203,4.835203


In [ ]:
wordlist = ' '.join(words['words'])
wordlist

"anthem chang use heal mini make skill buff minstrel like time class want play dp get melodi trait one group yellow would go even way effect need stanc see thing still raid damag also give new nerf blue realli actual coda pierc_cri remov think much induct red seem take yellow_line lot game reset someth i'm player current us 2 that' combat second better without sinc ballad disson good cooldown 3 tri instead less back everi mean keep point live line could feel great content work reduc cd differ peopl 1 mayb melodi_stanc wors long pleas reason landscap useless set alreadi anoth mani longer choic bad least never either fight problem well healer debuff alway 5 made say anyth anthem_third probabl main build two littl complet feel_like abl support look cri_choru come tier noth group_content said ssg special traceri rotat best know call_great activ red_line durat end feedback next bit benefit tank hope run red_mini must right hunter kind hit understand put start stun trait_tree can't plu insta

In [ ]:
sentences[0]

['hi',
 'everyon',
 'im',
 'onnmacmah',
 'want',
 'talk',
 'upcom_chang',
 'minstrel',
 'class',
 'weve',
 'convert',
 'minstrel',
 'protector_song',
 'special_tree',
 'passiv',
 'tree',
 'function',
 'way',
 'brawler',
 'yellow_tree',
 'im',
 'answer_question',
 'front',
 'give',
 'bit',
 'insight',
 'think',
 'regard_chang',
 'u33',
 '2',
 'minstrel',
 'longer',
 'abl_choos',
 'protector_song',
 'trait_tree',
 'special',
 'youll',
 'still',
 'abl',
 'special',
 'two',
 'trait_tree',
 'yellow_tree',
 'isnt',
 'go',
 'anywher',
 'wont',
 'abl_choos',
 'yellow_special',
 'still_full',
 'excit',
 'stuff',
 'player',
 'mean',
 'youll',
 'abl',
 'access_uniqu',
 'skill',
 'like',
 'song_aid',
 'call_great',
 'special',
 'either',
 'watcher_resolv',
 'warrior_skald',
 'trait_line',
 'yellow',
 'minstrel',
 'line',
 'strong_ident',
 'good_reason',
 'special',
 'protector_song',
 'reason',
 'arent',
 'usual',
 'better',
 'reason_special',
 'blue_red',
 'yet',
 'still',
 'clear',
 'util',
 're

In [ ]:
for index, value in enumerate(sentences):
    total_sent_val = 0.0
    # print(value)
    # print(index)
    for v in value:
        # print(v)
        if v in ' '.join(words['words']):
            score = words['sentiment_coeff'].iloc[np.where(v == words['words'])]
            # print(score)
            if len(score) > 0:
                # print(score)
                total_sent_val = total_sent_val + score.values
            # print(total_sent_val)
    print(total_sent_val)
    sentiment_arr[index] = total_sent_val[0]

[-146.93743849]
[-86.70122059]
[-81.73347438]
[-40.82361407]
[-68.54909867]
[-56.86453989]
[-42.23453153]
[-23.62761404]
[-172.77166892]
[-341.59707893]
[-61.88917054]
[-36.63441086]
[-405.96818517]
[-125.32401075]
[-1.89180695]
[-62.41099865]
[-99.17288163]
[-283.44709029]
[-198.89452069]
[-120.57032026]
[-84.10759545]
[-95.99796062]
[-206.21235541]
[-405.2525022]
[-53.35224026]
[-79.74373675]
[-2445.00938374]
[-58.87831245]
[-138.96581513]
[-409.91789417]
[-117.57285105]
[-1043.85101684]
[-201.71373545]
[-27.3297001]
[-112.31106518]
[-337.58058184]
[-222.45145305]
[-256.20716254]
[-36.81076785]
[-5.21580739]
[-136.88392275]
[-262.88933173]
[-296.03857852]
[-200.32801136]
[-345.36077419]
[-924.72634311]
[-169.39204044]
[-96.73802177]
[-72.81182765]
[-62.82452149]
[-201.57344444]
[-85.61122423]
[-146.03957104]
[-1514.6477681]
[-91.71480957]
[-35.43904423]
[-29.40281883]
[-10.48548941]
[-840.62597326]
[-624.4524567]
[-92.11687382]
[-173.45776312]
[-64.55032431]
[-1398.08154349]
[-50.011

In [ ]:
sentiment_arr

array([-1.46937438e+02, -8.67012206e+01, -8.17334744e+01, -4.08236141e+01,
       -6.85490987e+01, -5.68645399e+01, -4.22345315e+01, -2.36276140e+01,
       -1.72771669e+02, -3.41597079e+02, -6.18891705e+01, -3.66344109e+01,
       -4.05968185e+02, -1.25324011e+02, -1.89180695e+00, -6.24109986e+01,
       -9.91728816e+01, -2.83447090e+02, -1.98894521e+02, -1.20570320e+02,
       -8.41075955e+01, -9.59979606e+01, -2.06212355e+02, -4.05252502e+02,
       -5.33522403e+01, -7.97437368e+01, -2.44500938e+03, -5.88783124e+01,
       -1.38965815e+02, -4.09917894e+02, -1.17572851e+02, -1.04385102e+03,
       -2.01713735e+02, -2.73297001e+01, -1.12311065e+02, -3.37580582e+02,
       -2.22451453e+02, -2.56207163e+02, -3.68107679e+01, -5.21580739e+00,
       -1.36883923e+02, -2.62889332e+02, -2.96038579e+02, -2.00328011e+02,
       -3.45360774e+02, -9.24726343e+02, -1.69392040e+02, -9.67380218e+01,
       -7.28118277e+01, -6.28245215e+01, -2.01573444e+02, -8.56112242e+01,
       -1.46039571e+02, -

In [ ]:
corpus['sentiment'] = sentiment_arr
corpus

,post,sentiment
0,"[hi, everyone, im, onnmacmahal, and, i, want, ...",-146.937438
1,"[you're, deleting, the, classic, support, bard...",-86.701221
2,"[i, think, this, is, a, great, thought, !, thi...",-81.733474
3,"[i'm, worrying, that, this, will, decrease, th...",-40.823614
4,"[why, can't, anthems, be, made, available, out...",-68.549099
...,...,...
193,"[posting, feedback, in, this, thread, is, like...",-4063.826112
194,"[i, was, ready, to, be, super, upset, that, my...",-426.879451
195,"[well, dps, is, better, by, a, wide, margin, i...",-486.075068
196,"[if, anything, i, imagine, landscape, is, the,...",-142.439119


In [ ]:
corpus['avg_sent'] = corpus['sentiment']/len(corpus['post'])

In [ ]:
corpus.sort_values(by=['avg_sent']).head(20)

,post,sentiment,avg_sent
193,"[posting, feedback, in, this, thread, is, like...",-4063.826112,-20.628559
98,"[most, of, the, comments, i, would, write, her...",-2603.583145,-13.216158
66,"[hey, hey, !, i, am, super, excited, and, glad...",-2496.660614,-12.673404
27,"[piercing, cry, no, longer, stunning, in, diss...",-2445.009384,-12.411215
124,"[what, about, dissonance's, identity, ?, why, ...",-2296.067716,-11.655166
115,"[first, off, let, me, say, thank, you, so, muc...",-1819.762932,-9.237375
67,"[you, can't, use, anthems, out, of, combat, an...",-1553.280392,-7.884672
54,"[i, also, am, one, of, those, players, that, d...",-1514.647768,-7.688567
100,"[based, on, what, i, can, tell, the, plan, is,...",-1491.720078,-7.572183
169,"[hi, guys, i, just, want, to, share, some, tho...",-1489.857029,-7.562726


In [ ]:
df['post'][115]

'First off let me say thank you so much for your time and your feedback. I know changes like these can be frustrating but I really do appreciate all of you taking the time to read through the changes hop into Bullroarer and write out your feeling and feedback here.While I cant address everything thats been said here Im going to do my best to respond to some of the most common comments.Lets start off with Piercing Cry. While Piercing Cry is a delightful and iconic part of the minstrels toolkit the base effects of this skill were simply overloaded. Keep in mind this is a skill with a 10s cooldown and that cooldown can be reset as frequently as every 1 or 2 seconds. For a skill that a red-line minstrel will often be using between every other skill it was simply doing too much. Its immediate so it cancels animations from other skills. Its an interrupt so it will cancel monster inductions the second they start. It cleansed corruptions making it nearly impossible for a monster to build any c

Generating sentiment score via vader lexicon...

In [ ]:
vader = pd.read_csv(PATH+"vader_lexicon.txt", sep='\t', header=None)

In [ ]:
valence_vader = dict(vader[[0, 1]].values)

In [ ]:
valence_vader['great']

3.1

In [ ]:
modifiers = {
    "very": 1.5,
    "much": 1.3,
    "not": -1,
    "pretty": 1.5,
    "somewhat": 1.2
}

In [ ]:
def extract_words(text):
    temp = text.split() # Split the text on whitespace
    text_words = []

    for word in temp:
        if len(word) > 1:
            # Remove any punctuation characters present in the beginning of the word
            while word[0] in string.punctuation:
                word = word[1:]

            # Remove any punctuation characters present in the end of the word
            while word[-1] in string.punctuation:
                word = word[:-1]

            # Append this word into our list of words.
            text_words.append(word.lower())
        
    return text_words

In [ ]:
def sentiment_modified(text, valence, modifiers, verbose=False):
    words = extract_words(text.lower())
    
    ngrams = [[]]
    
    # generate ngrams
    for i in range(len(words)):
        word = words[i]
        
        if word in modifiers:
            ngrams[-1].append(word)
            continue
        if word in valence:
            ngrams[-1].append(word)
        else:
            if len(ngrams[-1]) > 0:
                ngrams.append([])
        # print(ngrams)

    score = 0 
    
    # Remove the trailing empty ngram if necessary
    if len(ngrams[-1]) == 0:
        ngrams = ngrams[:-1]

    for ngram in ngrams:
        value = 1

        for word in ngram:
            if word in modifiers:
                value *= modifiers[word]
            elif word in valence:
                value *= valence[word]

        if verbose:
            print(ngram, value)

        score += value
        
    return score/len(ngrams)

In [ ]:
"I am not very happy"

In [ ]:
sentiment_modified("I am not very happy", valence_vader, modifiers, verbose=True)

NameError: name 'sentiment_modified' is not defined

In [ ]:
corpus['vader_sentiment'] = 0.0

In [ ]:
for index in range(corpus.shape[0]+1):
    if index == 9:
        continue
    try:
        corpus['vader_sentiment'][index] = sentiment_modified(' '.join(corpus['post'][index]), valence_vader, modifiers)
    except:
        corpus['vader_sentiment'][index] = 0.0

C:\Users\sixbo\AppData\Local\Temp\ipykernel_20464\19520933.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus['vader_sentiment'][index] = sentiment_modified(' '.join(corpus['post'][index]), valence_vader, modifiers)
C:\Users\sixbo\AppData\Local\Temp\ipykernel_20464\19520933.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus['vader_sentiment'][index] = 0.0


In [ ]:
corpus

,post,sentiment,avg_sent,vader_sentiment
0,"[hi, everyone, im, onnmacmahal, and, i, want, ...",-146.937438,-0.745875,0.542055
1,"[you're, deleting, the, classic, support, bard...",-86.701221,-0.440108,0.406000
2,"[i, think, this, is, a, great, thought, !, thi...",-81.733474,-0.414891,1.440000
3,"[i'm, worrying, that, this, will, decrease, th...",-40.823614,-0.207226,-0.466667
4,"[why, can't, anthems, be, made, available, out...",-68.549099,-0.347965,-0.366667
...,...,...,...,...
193,"[posting, feedback, in, this, thread, is, like...",-4063.826112,-20.628559,-0.155843
194,"[i, was, ready, to, be, super, upset, that, my...",-426.879451,-2.166901,0.497647
195,"[well, dps, is, better, by, a, wide, margin, i...",-486.075068,-2.467386,0.579091
196,"[if, anything, i, imagine, landscape, is, the,...",-142.439119,-0.723041,0.125000


In [ ]:
corpus.iloc[np.where(corpus['vader_sentiment'] == 0.0)].sort_values(by=['vader_sentiment'])

,post,sentiment,avg_sent,vader_sentiment
34,"[so, when, can, i, expect, ssg, to, refund, th...",-27.329700,-0.138729,0.0
49,"[i'm, only, guessing, but, how, does, the, rai...",-72.811828,-0.369603,0.0
58,"[oooo, you're, right, i, should, buy, some, po...",-10.485489,-0.053226,0.0
66,"[hey, hey, !, i, am, super, excited, and, glad...",-2496.660614,-12.673404,0.0
97,"[first, off, all, good, morning, !, !, and, wo...",-285.881734,-1.451176,0.0
137,"[is, the, nerf, to, anthem, of, composure, tac...",-31.176352,-0.158256,0.0
161,"[do, you, actually, realise, that, minstrel, w...",-20.610673,-0.104623,0.0
